# Expresiones Regulares: De Cero a Maestro en NLP

Las expresiones regulares (Regex) son la "navaja suiza" del procesamiento de texto. Permiten buscar, extraer y validar patrones complejos con una precisión quirúrgica.

## Mapa de Ruta
1.  **Básicos**: Literales, Sets y Rangos.
2.  **Cuantificadores**: Greedy vs Lazy (Importante).
3.  **Grupos**: Captura, Grupos Nombrados y Estructuras.
4.  **Avanzado**: Lookarounds (Mirar sin tocar).
5.  **Workshop**: Parseo de Logs reales.

---

In [2]:
import re
import pandas as pd

# Función auxiliar para visualizar coincidencias
def mostrar_matches(patron, texto):
    matches = list(re.finditer(patron, texto))
    if not matches:
        print("No se encontraron coincidencias.")
        return
    
    print(f"Patrón: '{patron}' | Coincidencias encontradas: {len(matches)}")
    for i, m in enumerate(matches, 1):
        print(f"MATCH {i}: {m.group(0)!r} (Pos: {m.start()}-{m.end()})")
        if m.groupdict():
            print(f"   --> Grupos Nombrados: {m.groupdict()}")
        elif m.groups():
            print(f"   --> Grupos: {m.groups()}")
        print("-" * 30)

## 1. Fundamentos: Sets y Clases
- `.` : Cualquier caracter (menos nueva línea)
- `\d`: Dígitos [0-9]
- `\w`: Alfanumérico [a-zA-Z0-9_]
- `\s`: Espacio en blanco (espacio, tab, nueva línea)
- `[]`: Conjunto (Set). `[aeiou]` coincide con una vocal.

In [3]:
texto_basico = "El agente 007 vive en la Calle Falsa 123. Tel: 555-0199"

# Buscar números de 3 dígitos
mostrar_matches(r"\d{3}", texto_basico)

# Buscar palabras que empiezan con mayúscula (simple)
mostrar_matches(r"[A-Z]\w*", texto_basico)

Patrón: '\d{3}' | Coincidencias encontradas: 4
MATCH 1: '007' (Pos: 10-13)
------------------------------
MATCH 2: '123' (Pos: 37-40)
------------------------------
MATCH 3: '555' (Pos: 47-50)
------------------------------
MATCH 4: '019' (Pos: 51-54)
------------------------------
Patrón: '[A-Z]\w*' | Coincidencias encontradas: 4
MATCH 1: 'El' (Pos: 0-2)
------------------------------
MATCH 2: 'Calle' (Pos: 25-30)
------------------------------
MATCH 3: 'Falsa' (Pos: 31-36)
------------------------------
MATCH 4: 'Tel' (Pos: 42-45)
------------------------------


## 2. Greedy vs Lazy (Codicioso vs Perezoso)
Por defecto, `*` y `+` intentan comerse **todo lo que pueden**. Esto es peligroso en HTML o delimitadores.

In [4]:
texto_html = "<div>Titulo</div> <p>Parrafo</p>"

# GREEDY: Desde el primer < hasta el ÚLTIMO >
print("INTENTO GREEDY (<.*>):")
mostrar_matches(r"<.*>", texto_html)

# LAZY: Desde el primer < hasta el PRIMER > que encuentre
print("\nINTENTO LAZY (<.*?>):")
mostrar_matches(r"<.*?>", texto_html)

INTENTO GREEDY (<.*>):
Patrón: '<.*>' | Coincidencias encontradas: 1
MATCH 1: '<div>Titulo</div> <p>Parrafo</p>' (Pos: 0-32)
------------------------------

INTENTO LAZY (<.*?>):
Patrón: '<.*?>' | Coincidencias encontradas: 4
MATCH 1: '<div>' (Pos: 0-5)
------------------------------
MATCH 2: '</div>' (Pos: 11-17)
------------------------------
MATCH 3: '<p>' (Pos: 18-21)
------------------------------
MATCH 4: '</p>' (Pos: 28-32)
------------------------------


## 3. Grupos Nombrados: La forma Profesional
En lugar de contar paréntesis `(Grupo 1, Grupo 2...)`, ponles nombre: `(?P<nombre>patron)`.

In [5]:
fecha = "2023-12-25"

# Patrón para extraer Año, Mes, Día limpiamente
patron_fecha = r"(?P<anio>\d{4})-(?P<mes>\d{2})-(?P<dia>\d{2})"

mostrar_matches(patron_fecha, fecha)

Patrón: '(?P<anio>\d{4})-(?P<mes>\d{2})-(?P<dia>\d{2})' | Coincidencias encontradas: 1
MATCH 1: '2023-12-25' (Pos: 0-10)
   --> Grupos Nombrados: {'anio': '2023', 'mes': '12', 'dia': '25'}
------------------------------


## 4. Lookarounds (Magia Avanzada)
Permiten validar condiciones sin "consumir" el texto. 
- `(?=...)` Positive Lookahead (Debe haber X después)
- `(?!...)` Negative Lookahead (NO debe haber X después)
- `(?<=...)` Positive Lookbehind (Debe haber X antes)
- `(?<!...)` Negative Lookbehind (NO debe haber X antes)

In [6]:
precios = "Cuesta $100 pero la oferta es 90 USD y antes solía costar 120€"

# Queremos números que tengan el símbolo $ ANTES (Lookbehind)
print("--- Lookbehind: Numeros precedidos de $ ---")
mostrar_matches(r"(?<=\$)\d+", precios)

# Queremos números seguidos de ' USD' (Lookahead)
print("\n--- Lookahead: Numeros seguidos de USD ---")
mostrar_matches(r"\d+(?= USD)", precios)

--- Lookbehind: Numeros precedidos de $ ---
Patrón: '(?<=\$)\d+' | Coincidencias encontradas: 1
MATCH 1: '100' (Pos: 8-11)
------------------------------

--- Lookahead: Numeros seguidos de USD ---
Patrón: '\d+(?= USD)' | Coincidencias encontradas: 1
MATCH 1: '90' (Pos: 30-32)
------------------------------


## 5. Workshop: Log Parsing Real
Imagina que tienes logs de servidor desordenados.

In [ ]:
log_data = """
192.168.1.1 - [12/Oct/2023:14:00:01] "GET /index.html HTTP/1.1" 200 1024
10.0.0.5 - [12/Oct/2023:14:01:45] "POST /login HTTP/1.1" 403 512
"""

# Haremos un patrón robusto con re.VERBOSE para poder comentarlo
patron_log = r"""
    (?P<ip>\d+\.\d+\.\d+\.\d+)       # Captura la IP
    \s-\s                           # Separador guion
    \[(?P<fecha>.*?)\]              # Fecha entre corchetes (Lazy)
    \s
    "(?P<metodo>\w+)                # Metodo HTTP (GET/POST)
    \s
    (?P<ruta>.*?)                   # Ruta (Lazy)
    \s
    HTT.*"                          # Resto del protocolo
    \s
    (?P<status>\d{3})               # Codigo de status (3 digitos)
"""

matches = re.finditer(patron_log, log_data, re.VERBOSE)

datos_estructurados = [m.groupdict() for m in matches]
df_logs = pd.DataFrame(datos_estructurados)
df_logs